In [1]:
import json
import math
import numpy as np
import cvxpy as cp
import csv
import pandas as pd

# used to extend all 1s
np.set_printoptions(threshold=np.inf)
def past_term(matrix):
    result_number=0
    for element in matrix:
        flag=True
        for i in range(0,len(element)):
            if element[i]!="0.0":
               # print(i)
                if flag == True:
                    flag = False
                    if i>result_number:
                        result_number=i
    #print(result_number)
    return result_number

def number_checker(lst):
    temp_number=''
    for i in lst:
        if i=='.':
            return temp_number
        if i.isdigit():
            temp_number=temp_number+i
    return temp_number
                
def at_least_course(lst):
    result=""
    add=False
    result_list=[]
    for element in lst:
        if element=='[':
            add=True
            #result=result+element
        elif element==']':
           # result=result+element
            add=False
            result_list=result.split(',')
            return result_list
        else:
            if add==True:
                result=result+element
                
def number_of_course(lst):
    check=False
    result=""
    for element in lst:
        if element == ']':
            check=True
        if check==True:
            if element.isdigit():
                result=result+str(element)
    return result
                
def display(lst,num):
    total=0
    counter=1
    for element in lst:
        if element[num]==1:
            print(counter)
            print(element)
        counter+=1
        total+=element[num]
    return total
        
                
# student file data processing
with open("/Users/sebastianjin/Documents/Berkeley/plan/9.csv",'r') as f:
    csvreader = list(csv.reader(f))
    whole_matrix = np.array(csvreader[1:])
    whole_matrix = whole_matrix.astype(np.float64)
    row,column=whole_matrix.shape
    # all probability matrices we need 
    probability_matrix = whole_matrix[:,1:]
   
    # history courses we need
    past_course=whole_matrix[:,1:4]
    past_course_2= whole_matrix[:,1:4]
    remaining_matrix=np.zeros((row,6))
    
    print(past_course_2)
    
    past_course=np.append(past_course,remaining_matrix,axis=1)

# total number of courses already taken for the first 3 semesers
total=0
for element in past_course:
    total+=int(element[0])
    total+=int(element[1])
    total+=int(element[2])

# course data processing into dictinary
with open("/Users/sebastianjin/Documents/Berkeley/Cortland_Courses.csv",'r') as f:
    csvreader = list(csv.reader(f))
    csvreader=csvreader[1:]
    course_dict={}
    course_ref={}
    for element in csvreader:
        course_dict[element[1]]=int(element[0])-1
        course_ref[int(element[0])-1]=element[1]

        
# read in json requirement file
with open("/Users/sebastianjin/Documents/Berkeley/criminology_requirement.json",'r') as f:
    data = json.load(f)
    least_credit=0
    residency_credit=0
    gpa=0
    total_class={}
    i=0
    total_course=0
    for lines in data:
        
        if "Credits >=" in lines and lines[0]=='C':
            least_credit=number_checker(lines)
            
        if "ResidenceCredits >=" in lines:
            residency_credit=number_checker(lines)
            
        if "GPA >=" in lines:
            gpa=number_checker(lines)
 
# credits 1857 by 8 all 3 credits
credits=np.full((past_course.shape[0],past_course.shape[1]), 3, dtype=int)

# schedule matrix 1857 by 8 0's and 1's
schedule=np.ones((past_course.shape[0],past_course.shape[1]),dtype=int)

# add missing courses to student file and course file with 0's
# try except to catch errors
priority_list=[['CRM222'],['CRM373', 'SOC373'],['CRM463', 'SOC463'],['SOC150'],['SOC301', 'SOC493', 'SOC293'],['SOC302', 'SOC494', 'SOC294'],
              ['SOC300'],['SOC230', 'SOC235', 'SOC315', 'SOC322', 'SOC335', 'SOC340', 'SOC350', 'SOC351', 'SOC352', 'SOC355', 'SOC360', 'SOC366', 'SOC370', 'SOC371', 'SOC385', 'SOC401', 'SOC430', 'SOC451', 'SOC465', 'SOC470', 'SOC475', 'SOC480', 'SOC505'],
              ['CRM202', 'CRM303', 'CRM375', 'POL202', 'SOC375'],['CRM333', 'CRM363', 'CRM392', 'CRM464', 'CRM477', 'CRM485', 'PHI205'],['CRM380', 'CRM383'],['CRM325', 'CRM333', 'CRM345', 'CRM348', 'CRM363', 'CRM365', 'CRM372', 'CRM373', 'CRM377', 'CRM378', 'CRM380', 'CRM383', 'CRM392', 'CRM395', 'CRM399', 'CRM403', 'CRM404', 'CRM412', 'CRM462', 'CRM463', 'CRM464', 'CRM477', 'CRM485']]
#print(course_dict)
required_list=[]
for element in priority_list:
    temp=[]
    for element_2 in element:
        temp.append(course_dict[element_2])
    required_list.append(temp)

# this is the output matrix
result=cp.Variable((past_course.shape[0],past_course.shape[1]),boolean=True)

# semester credits limit
semester_credit_matrix=cp.multiply(credits,result)

# total credits suggested
total_credits=cp.sum(cp.multiply(result,credits))

# non repeat constraint
non_repeat=np.ones(result.shape[0])

#loss constraint
total_prob=cp.sum(cp.multiply(result,probability_matrix))
total_courses=cp.sum(result)
loss=total_courses-total_prob
# requirements
for i in range(0,len(required_list)):
    if i == 0 :
        requirement_1=0
        for b in required_list[i]:
            requirement_1+=cp.sum(result[b-1])
    if i == 1 :
        requirement_2=0
        for b in required_list[i]:
            requirement_2+=cp.sum(result[b-1])
    if i == 2 :
        requirement_3=0
        for b in required_list[i]:
            requirement_3+=cp.sum(result[b-1])
    if i == 3 :
        requirement_4=0
        for b in required_list[i]:
            requirement_4+=cp.sum(result[b-1])
            
    if i == 4 :
        requirement_5=0
        for b in required_list[i]:
            requirement_5+=cp.sum(result[b-1])
            
    if i == 5 :
        requirement_6=0
        for b in required_list[i]:
            requirement_6+=cp.sum(result[b-1])
    
    if i == 6 :
        requirement_7=0
        for b in required_list[i]:
            requirement_7+=cp.sum(result[b-1])
    if i == 7 :
        requirement_8=0
        for b in required_list[i]:
            requirement_8+=cp.sum(result[b-1])
    if i == 8 :
        requirement_9=0
        for b in required_list[i]:
            requirement_9+=cp.sum(result[b-1])
    if i == 9 :
        requirement_10=0
        for b in required_list[i]:
            requirement_10+=cp.sum(result[b-1])
            
    if i == 10 :
        requirement_11=0
        for b in required_list[i]:
            requirement_11+=cp.sum(result[b-1])
            
    if i == 11 :
        requirement_12=0
        for b in required_list[i]:
            requirement_12+=cp.sum(result[b-1])

#first_history=past_course[:,0]
#second_history=past_course[:,1]
first_history=int(display(past_course,0))
second_history=int(display(past_course,1))
third_history=int(display(past_course,2))
ones_matrix=np.ones(past_course.shape[0])
#print(ones_matrix.shape)

# more parameter requirements ( finish the degree in X semesters)
# minimize the cost (maybe?)
prob=cp.Problem(cp.Minimize(loss),
                [
                  #  total credits requirement
                  #  total_credits>=least_credit,total_credits>=120,
                    total_credits>=120,
                    # course offering requirement
                    cp.multiply(schedule,result)==result,
                    
                    
                    # number of courses and credits limit
                    # needs improvements 
                    requirement_1>=1,requirement_2>=1,requirement_3>=1,
                    requirement_4>=1,requirement_5>=1,requirement_6>=1,
                    requirement_7>=1,requirement_8>=1,requirement_9>=1,
                    requirement_10>=2,requirement_11>=1,requirement_12>=2,
                    
                    # semester credits constraints
                    #cp.multiply(result[:,[0,1,2]],past_course_2)==past_course_2,
                    result[:,[0,1,2]]==past_course_2,
                    
                    # sum(past_course_2)=cp.sum
                   
                    cp.sum(semester_credit_matrix[:,3])>=12,cp.sum(semester_credit_matrix[:,3])<=18,
                    cp.sum(semester_credit_matrix[:,4])>=12,cp.sum(semester_credit_matrix[:,4])<=18,
                    cp.sum(semester_credit_matrix[:,5])>=12,cp.sum(semester_credit_matrix[:,5])<=18,
                    cp.sum(semester_credit_matrix[:,6])>=12,cp.sum(semester_credit_matrix[:,6])<=18,
                    cp.sum(semester_credit_matrix[:,7])>=12,cp.sum(semester_credit_matrix[:,7])<=18,
                    cp.sum(semester_credit_matrix[:,8])>=12,cp.sum(semester_credit_matrix[:,8])<=18,
                    
                    # non-repeat courses constrint
                    cp.sum(result,axis=1)<=ones_matrix,
                    
                    # remaining course constraint
                    cp.sum(result-past_course)>=12,
                    
                    # remaining credit constraint 
                    cp.sum(cp.multiply(credits,result-past_course))>=30,
                    
                ]
               )
prob.solve()
b=result.value
print(b)
print("Below is the sum of courses taken per semester")
print(sum(b))
#print((b[:,[0,1]]==past_course_2).all())
#print(b[:,[0,1]])
#print(past_course_2)
#print(sum(sum(b[:,[0,1]])))
#print(sum(sum((past_course_2))))
#total=0



# the code below is used to save your generated result in a csv file
DF = pd.DataFrame(b)
DF.columns=['term_0','term_1','term_2','term_3','term_4','term_5','term_6','term_7','term_8']
DF.index = np.arange(1, len(DF) + 1)
DF.to_csv("/Users/sebastianjin/Documents/Berkeley/plan/new_999.csv")


# next goals may 26th
# 1: developing and sharing code into a github (cal repo github)
#   
# 2: extend (incorporate) last sets of requirements (general degree requirements)

# semester 0 (AP, High school portion) 3 history courses in total.
# not everystudent has semester 0. 
# semesters passed in as parameters (including past history semesters)
# the semester creidts should also be dynamic parameters that are passed in. (user defined)




               
#def at_least_credits(lst,dictionary):
        

# read in json requirement file
# we have past course history past_course
# we have probability matrix probability
# we have course_dict for course reference

   

   
        
# add all missing courses in csv files (check for dictionary)
# add missing courses in both csv files and numpy matrices 
# add to both student files and course files
# probabilities all 0s
# history courses only appear in the first 2 columns
# 3-4 columns are used to take only courses that are able to transferred
# 5-8 are future columns 
# human readable outputs 





# google calendar invitations 
# what the program is (major name)
# any additional information about the email
# all day invite
# 


# goals for this week:
# output the result in the same format with 0's and 1's
# dynamically load in files 
# at least k credits and k courses constraints 
# process the requirement variables 

# maybe grades


# try debugging 9th file



# approach 1 
# add rows to be 0's for rest of the data (fault tolerance)
# fix the rows to be 0's
# approach 2
# pre-process the data 
# send back the result ASAP

(CVXPY) Jul 10 12:02:40 AM: Encountered unexpected exception importing solver SCS:
ImportError("dlopen(/Users/sebastianjin/anaconda3/lib/python3.10/site-packages/_scs_direct.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib\n  Referenced from: <EA245032-41C7-30EA-BA1A-C55492696949> /Users/sebastianjin/anaconda3/lib/python3.10/site-packages/_scs_direct.cpython-310-darwin.so\n  Reason: tried: '/Users/sebastianjin/anaconda3/lib/python3.10/site-packages/../../liblapack.3.dylib' (no such file), '/Users/sebastianjin/anaconda3/lib/python3.10/site-packages/../../liblapack.3.dylib' (no such file), '/Users/sebastianjin/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/sebastianjin/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)")
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [ ]:
# course data processing into dictinary
import json
import math
import numpy as np
#import cvxpy as cp
import csv
import pandas as pd
with open("/Users/sebastianjin/Documents/Cortland_Courses.csv",'r') as f:
    csvreader = list(csv.reader(f))
    csvreader=csvreader[1:]
    course_dict={}
    course_ref={}
    for element in csvreader:
        course_dict[element[1]]=int(element[0])-1
        course_ref[int(element[0])-1]=element[1]
        
# read in json requirement file
with open("/Users/sebastianjin/Documents/general_requirement.json",'r') as f:
    data = json.load(f)
    least_credit=0
    residency_credit=0
    gpa=0
    total_class={}
    i=0
    total_course=0
    priority_list=[]
    temp_string=1
    for lines in data:
        temp_list=[]
        temp_line=lines[12:len(lines)-1]
        for element in temp_line:
            if element=='[':
                temp_string=''
            else:
                if temp_string!=1:
                    
                    if element!=']':
                        temp_string=temp_string+element
                        #print(temp_string)
                    else:
                       
                        priority_list.append(temp_string.split(','))
                        temp_string=1
                        break
    print(priority_list)
    
    
    # take into account
    # try/except through error 
    # also pay attention to or statement (1st,2nd lines)
    # try to solve the "or" problem but not all of them
    # also support "and"
    # 
        
                        
                    
                
                
        
        
        

  Using cached cvxpy-1.3.1-cp310-cp310-macosx_10_9_universal2.whl (1.2 MB)
  Using cached osqp-0.6.3.tar.gz (228 kB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [107 lines of output]
        Using cached oldest_supported_numpy-2022.11.19-py3-none-any.whl (4.9 kB)
        Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
        Using cached wheel-0.40.0-py3-none-any.whl (64 kB)
        Using cached setuptools_scm-7.1.0-py3-none-any.whl (43 kB)
        Using cached qdldl-0.1.7.tar.gz (70 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Installing backend dependencies: started
        Installing backend dependencies: finished with status 'done'
  

Note: you may need to restart the kernel to use updated packages.


Retrieving notices: ...working... done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - cvxpy

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-arm64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-arm64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



## Package Plan ##

  environment location: /Users/sebastianjin/anaconda3

  added / updated specs:
    - cvxpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       hf0a4a13_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    cryptography-38.0.4        |  py310hfc83b78_0         1.0 MB  conda-forge
    cvxpy-1.3.1                |  py310hb6292c7_1         112 KB  conda-forge
    cvxpy-base-1.3.1           |  py310h1cdf563_1         982 KB  conda-forge
    ecos-2.0.11                |  py31

 ... (more hidden) ...
libpq-12.15          | 2.4 MB    | 2                                     |   1% 

cvxpy-1.3.1          | 112 KB    | #####3                                |  14% 



osqp-0.6.3           | 112 KB    | #####2                                |  14% 


cvxpy-1.3.1          | 112 KB    | ##################################### | 100% 




python_abi-3.10      | 6 KB      | ##################################### | 100% 


scs-3.2.3            | 69 KB     | ##################################### | 100% 




python_abi-3.10      | 6 KB      | ##################################### | 100% 





libosqp-0.6.3        | 63 KB     | #########4                            |  26% 



osqp-0.6.3           | 112 KB    | ##################################### | 100% 






liblapack-3.9.0      | 13 KB     | ##################################### | 100% 





libosqp-0.6.3        | 63 KB     | ##################################### | 100% 






liblapack-3.9.0      | 13 KB     | ##########

libqdldl-0.1.5       | 15 KB     | ##################################### | 100% 






















 ... (more hidden) ...






















 ... (more hidden) ...














cvxpy-base-1.3.1     | 982 KB    | ##################################### | 100% 

















openssl-3.1.1        | 2.1 MB    | ######################3               |  60% 


















cryptography-38.0.4  | 1.0 MB    | ##########8                           |  29% 




















python-3.10.12       | 11.9 MB   | ##5                                   |   7% 



















libcxx-16.0.6        | 1.1 MB    | ##################################### | 100% 


















cryptography-38.0.4  | 1.0 MB    | ##############2                       |  39% 




















python-3.10.12       | 11.9 MB   | ###8                                  |  10% 

















openssl-3.1.1        | 2.1 MB    | ###########################2          |  74% 


















cryptography-38.0.4 

In [ ]:
conda update -n base -c defaults conda